# Import modules

In [1]:
%matplotlib
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import pi,h
from qutip import *
import functools
import operator

Using matplotlib backend: TkAgg


# Two-level single qubit gate with SFQ pulse train

## Using a SFQ pulse train in H1(t)

In [ ]:
# check a Gaussian pulse train

# parameters
w_01 = 5 * 2 *pi # in GHz*2*pi 
tau = 2e-3 # in ns 
tc = 5*tau # in ns 
del_theta = 2*pi/360*2 # in rad
n = 90

t1 = np.linspace(0, n * 2*pi/w_01, 100001)
y1 = functools.reduce(operator.add, [ np.exp(-(t1-tc -i*2*pi/w_01)**2/2/tau**2) * (np.heaviside(t1-i*2*pi/w_01,0)- np.heaviside(t1-2*tc-i*2*pi/w_01,0)) for i in range(n)])

plt.plot(t1, y1)

In [2]:
def H1_coeff(t, args):
    out = functools.reduce(operator.add, 
        [np.exp(-(t-args['tc']-i*2*pi/args['w_01'])**2/2/args['tau']**2) 
         * (np.heaviside(t-i*2*pi/args['w_01'],0) + np.heaviside(t+2*args['tc']-i*2*pi/args['w_01'],0)) 
         for i in range(args['n'])])
    
    return out

In [ ]:
tlist_pulseTrain = np.linspace(0, n * 2*pi/w_01, 10001)
plt.plot(tlist_pulseTrain, H1_coeff(tlist_pulseTrain, args))

In [5]:
# parameters
w_01 = 5 * 2 *pi # in GHz*2*pi 
tau = 2e-3 # in ns 
tc = 5*tau # in ns 
del_theta = 2*pi/360*4 # in rad
n = 45

# solve Lindblad master equation
H0 = w_01/2*(qeye(2) - sigmaz()) 
H1 = del_theta/2/np.sqrt(2*pi)/tau*sigmay()
H = [H0, [H1, H1_coeff]]

psi0 = basis(2,0)
tlist_pulseTrain = np.linspace(0, n * 2*pi/w_01, 5001)
args = {'tau':tau, 'tc': tc, 'w_01': w_01,'n':n}

output = mesolve(H, psi0, tlist_pulseTrain, [], [sigmax(), sigmay(),sigmaz()], args, progress_bar=True)  
exp_x = output.expect[0]
exp_y = output.expect[1]
exp_z = output.expect[2]

# plot
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax.plot(tlist_pulseTrain, exp_z)
ax.set_xlabel('Time (ns)', fontsize=18)
ax.set_ylabel(r'$<\sigma_z>$', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

# # plot in Bloch sphere
bloch = Bloch()
bloch.add_points([exp_x, exp_y, exp_z], meth='l')
bloch.show()

10.0%. Run time:   3.21s. Est. time left: 00:00:00:28
20.0%. Run time:   7.03s. Est. time left: 00:00:00:28
30.0%. Run time:  10.46s. Est. time left: 00:00:00:24
40.0%. Run time:  14.00s. Est. time left: 00:00:00:20
50.0%. Run time:  17.13s. Est. time left: 00:00:00:17
60.0%. Run time:  20.07s. Est. time left: 00:00:00:13
70.0%. Run time:  23.48s. Est. time left: 00:00:00:10
80.0%. Run time:  26.76s. Est. time left: 00:00:00:06
90.0%. Run time:  30.02s. Est. time left: 00:00:00:03
Total run time:  33.24s


In [7]:
bloch.add_points?

## Using evolution operator : faster

In [ ]:
def H1_coeff_1(t, args):
    """ Time-dependant coefficient of H1 Hamiltonian """
    return np.exp(-t**2/2/args['tau']**2)

def U_f(t):
    """ free time evolution operator """
    return (1j*w_01*t/2*sigmaz()).expm() 

def U_single_multiplier(U_single, n):
    U = U_single
    counter = 1
    while True:
        if counter > n: return
        yield U 
        U *= U_single
        counter += 1

In [ ]:
# parameters
w_01 = 5 * 2 *pi # in GHz*2*pi 
tau = 4e-3 # in ns 
tc = 5*tau # in ns 
del_theta = 2*pi/360*2 # in rad
n = 180

# Compute evolution operator over one unit of SFQ pulse train,
# i.e., one SFQ + free evolotuion
H0 = w_01/2*(qeye(2) - sigmaz()) 
H1 = del_theta/2/np.sqrt(2*pi)/tau*sigmay() 
H = [H0, [H1, H1_coeff_1]]

tlist = np.linspace(-tc, tc, 101)
args = {'tau':tau, 'tc':tc, 'w_01':w_01}

U_single_Gauss = propagator(H, tlist, [], args=args) # time evolution during a Gaussian pulse

U_single = U_f(2*pi/w_01-tc) * U_single_Gauss[-1] * U_f(-tc) # Eq.26 in PRApplied paper.

U_G_list = [i for i in U_single_multiplier(U_single, n)]

# U_G_list = [U_single]
# for i in range(n-1):
#     U_G_list.append(U_single*U_G_list[-1])

sigmaz_expect = expect(sigmaz(), U_G_list*basis(2,0)) # expectation value of sigma_z
sigmay_expect = expect(sigmay(), U_G_list*basis(2,0))
sigmax_expect = expect(sigmax(), U_G_list*basis(2,0))

# plot
tlist = [2*pi/w_01*i for i in range(n)]
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax.plot(tlist, sigmaz_expect, label='<z>')
ax.plot(tlist, sigmay_expect, label='<y>')
ax.plot(tlist, sigmax_expect, label='<x>')
ax.set_xlabel('Time (ns)', fontsize=18)
ax.set_ylabel(r'$<\sigma_z>$', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')
ax.legend(fontsize=18)

In [ ]:
U_single_Gauss

## Fidelity vs Number of SFQ pulses 
As in Fig.4

In [ ]:
# parameters
w_01 = 5 * 2 *pi # in GHz*2*pi 
tau = 4e-3 # in ns 
tc = 5*tau # in ns 
theta = pi/2

H0 = w_01/2*(qeye(2) - sigmaz()) 
U_id = (-1j*theta*sigmay()/2).expm()

N_list = np.arange(1, 300, 1)
fid_list = []
UG_list = []
for n in N_list:
    del_theta = theta / float(n)
    H1 = del_theta/2/np.sqrt(2*pi)/tau*sigmay() 
    H = [H0, [H1, H1_coeff_1]]
    
    tlist = np.linspace(-tc, tc, 101)
    args = {'tau':tau, 'tc':tc, 'w_01':w_01}

    U_single_Gauss = propagator(H, tlist, [], args=args) # time evolution during a Gaussian pulse

    U_single = U_f(2*pi/w_01-tc) * U_single_Gauss[-1] * U_f(-tc) # Eq.26 in PRApplied paper.

    U_G = U_single**n
    UG_list.append(U_G) # optional. for debugging
    
    Fid_n = (np.absolute((U_id.dag()*U_G).tr())**2 + 2) / 6
    fid_list = np.append(fid_list,Fid_n)


# plot
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax.plot(N_list, 1 - fid_list)
ax.set_xlabel('Number of SFQ pulses', fontsize=18)
ax.set_ylabel('Gate error, 1-F', fontsize=18)
ax.set_xscale('log')
ax.set_yscale('log')
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

# Two-level two-qubit gate with CR
Easwar Eq. 3.1

In [13]:
# parameters
w1 = 5.2 * 2 *pi # in GHz*2*pi 
w2 = 5 * 2 * pi  # target Q
J = 0.003*2*pi
Omega = 0.05*2*pi
phi = 0 * 2 * pi

b = destroy(2)

# solve Lindblad master equation
H0 = w1*tensor(b.dag()*b, qeye(2)) + w2*tensor(qeye(2), b.dag()*b) \
    + J*(tensor(b.dag(), b) + tensor(b, b.dag()))
H1 = tensor((b.dag() + b), qeye(2))
H1_coeff = lambda t, args: args['Omega'] * np.cos(args['w2']*t + args['phi'])
H = [H0, [H1, H1_coeff]]

psi0_g = tensor(basis(2,0), basis(2,0))
psi0_e = tensor(basis(2,1), basis(2,0))
# tlist = np.linspace(0, 1000, 5001)  # ns
tlist = np.arange(0, 2500, 1)  # ns. if 0.5 is used, there is fast oscillation. why?
args = {'Omega':Omega, 'w2': w2, 'phi':phi}

# output_g = mesolve(H, psi0_g, tlist, [], [tensor(qeye(2), sigmaz()),tensor(qeye(2), sigmax()),tensor(qeye(2), sigmay())], args, progress_bar=True)  
# output_e = mesolve(H, psi0_e, tlist, [], [tensor(qeye(2), sigmaz()),tensor(qeye(2), sigmax()),tensor(qeye(2), sigmay())], args, progress_bar=True)  
U = propagator(H, tlist, [], args=args, progress_bar=True)
x_g = expect(tensor(qeye(2), sigmax()), U*psi0_g)
y_g = expect(tensor(qeye(2), sigmay()), U*psi0_g)
z_g = expect(tensor(qeye(2), sigmaz()), U*psi0_g)

x_e = expect(tensor(qeye(2), sigmax()), U*psi0_e)
y_e = expect(tensor(qeye(2), sigmay()), U*psi0_e)
z_e = expect(tensor(qeye(2), sigmaz()), U*psi0_e)

# in rotating frame
R = (-1j*(tensor(b.dag()*b, qeye(2)) + tensor(qeye(2), b.dag()*b))).expm()
R*U*psi0_g

# plot
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax.plot(tlist, x_g, tlist,y_g, tlist, z_g)
ax.plot(tlist, x_e, tlist,y_e, tlist, z_e)
ax.set_xlabel('Time (ns)', fontsize=18)
ax.set_ylabel(r'$<\sigma_z>$', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

# plot in Bloch sphere
bloch = Bloch()
bloch.add_points([x_g, y_g, z_g], meth='l')
bloch.add_points([x_e, y_e, z_e], meth='l')
bloch.show()

In [14]:
U

array([Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]],
       Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 9.88799554e-01+4.56005726e-03j -1.31546012e-03+5.35383619e-04j
  -8.76821917e-02-1.20683397e-01j  4.47211113e-05+6.15503756e-05j]
 [-1.31545943e-03+5.35383539e-04j  9.88644874e-01+4.62812187e-03j
  -1.02750174e-02-1.41423240e-02j -8.76870078e-02-1.20680004e-01j]
 [-8.76821620e-02-1.20683319e-01j -1.02750163e-02-1.41423229e-02j
   3.01105215e-01-9.41687529e-01j -9.15654659e-04+1.08567396e-03j]
 [ 4.47208202e-05+6.15496912e-05j -8.76870411e-02-1.20678407e-01j
  -9.15643589e-04+1.08566806e-03j  3.01128459e-01-9.41842319e-01j]],
       Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 9.70828921e-01+3.01801081e-02j -3.40847213e-03+3.51586654e-03j
  -2.26176477e-01-7.3487675

In [ ]:
# Diagonalize and then apply CR drving
# The result looks the same as without diagonalization
def sorted_diag_op(eigen):
    """
    Diagonalizing matrix by the order of eigenvector
    
    Args:
        eigen : output of eigenstates() method in qutip
       
    Return: a new diagonalizing operator Qobj
    """
       
    index_list = [np.argmax(np.absolute(vec)) for vec in eigen[1]]
    
    X_array = np.column_stack([eigen[1][index_list.index(i)] for i in range(eigen[1].size)])
    X = Qobj(X_array)
    
    return X

# parameters
w1 = 5.2 * 2 *pi # in GHz*2*pi 
w2 = 5 * 2 * pi  # target Q
J = 0.003*2*pi
Omega = 0.05*2*pi
phi = 0 * 2 * pi

b = destroy(2)

# solve Lindblad master equation
H0 = w1*tensor(b.dag()*b, qeye(2)) + w2*tensor(qeye(2), b.dag()*b) \
    + J*(tensor(b.dag(), b) + tensor(b, b.dag()))
eigen = H0.eigenstates()

# find U
U = sorted_diag_op(eigen)
U.dims = H0.dims

# digonalize H0
H0_d = U.dag() * H0 * U

H1 = U.dag()*tensor((b.dag() + b), qeye(2))*U
H1_coeff = lambda t, args: args['Omega'] * np.cos(args['w2']*t + args['phi'])
H = [H0_d, [H1, H1_coeff]]

psi0_g = tensor(basis(2,0), basis(2,0))
psi0_e = tensor(basis(2,1), basis(2,0))
# tlist = np.linspace(0, 1000, 5001)  # ns
tlist = np.arange(0, 2500, 1)  # ns if 0.5 is used, there is fast oscillation. why?
args = {'Omega':Omega, 'w2': w2, 'phi':phi}

# output_g = mesolve(H, psi0_g, tlist, [], [tensor(qeye(2), sigmaz()),tensor(qeye(2), sigmax()),tensor(qeye(2), sigmay())], args, progress_bar=True)  
# output_e = mesolve(H, psi0_e, tlist, [], [tensor(qeye(2), sigmaz()),tensor(qeye(2), sigmax()),tensor(qeye(2), sigmay())], args, progress_bar=True)  
U = propagator(H, tlist, [], args=args, progress_bar=True)
x_g = expect(tensor(qeye(2), sigmax()), U*psi0_g)
y_g = expect(tensor(qeye(2), sigmay()), U*psi0_g)
z_g = expect(tensor(qeye(2), sigmaz()), U*psi0_g)

x_e = expect(tensor(qeye(2), sigmax()), U*psi0_e)
y_e = expect(tensor(qeye(2), sigmay()), U*psi0_e)
z_e = expect(tensor(qeye(2), sigmaz()), U*psi0_e)

# plot
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax.plot(tlist, x_g, tlist,y_g, tlist, z_g)
ax.plot(tlist, x_e, tlist,y_e, tlist, z_e)
ax.set_xlabel('Time (ns)', fontsize=18)
ax.set_ylabel(r'$<\sigma_z>$', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

# plot in Bloch sphere
bloch = Bloch()
bloch.add_points([x_g, y_g, z_g])
bloch.add_points([x_e, y_e, z_e])
bloch.show()

In [ ]:
# Easwar paper, Eq.3.16, two-level qubit with CR drive. Ideal case.
Delta = 0.2 *2*pi
Omega = 0.02*2*pi
J = 0.005*2*pi

tlist = np.arange(0, 2000, 1)  # ns
H0 = (Delta - np.sqrt(Delta**2 + Omega**2))/2*tensor(sigmaz(),qeye(2)) \
      - (J*Omega/np.sqrt(Delta**2 + Omega**2))/2*tensor(sigmaz(), sigmax())
U = propagator(H0, tlist, [])

psi0_g = tensor(basis(2,0), basis(2,0))
psi0_e = tensor(basis(2,1), basis(2,0))

x_g = expect(tensor(qeye(2), sigmax()), U*psi0_g)
y_g = expect(tensor(qeye(2), sigmay()), U*psi0_g)
z_g = expect(tensor(qeye(2), sigmaz()), U*psi0_g)

x_e = expect(tensor(qeye(2), sigmax()), U*psi0_e)
y_e = expect(tensor(qeye(2), sigmay()), U*psi0_e)
z_e = expect(tensor(qeye(2), sigmaz()), U*psi0_e)

bloch = Bloch()
bloch.add_points([x_g, y_g, z_g])
bloch.add_points([x_e, y_e, z_e])
bloch.show()

In [ ]:
(w1*tensor(b.dag()*b, qeye(2)) + w2*tensor(qeye(2), b.dag()*b) )/2/pi

In [ ]:
H0

# Two-level two-qubit gate with CR using sigmaz

In [ ]:
# parameters
w1 = 5.2 * 2 *pi # in GHz*2*pi 
w2 = 5 * 2 * pi  # target Q
J = 0.003*2*pi
Omega = 0.05*2*pi
phi = 0 * 2 * pi

sp = sigmap()
sm = sigmam()

# solve Lindblad master equation
H0 = w1/2*tensor(sigmaz(), qeye(2)) + w2/2*tensor(qeye(2), sigmaz()) \
    + J*(tensor(sp, sm) + tensor(sm, sp))
H1 = tensor(sigmax(), qeye(2))
H1_coeff = lambda t, args: args['Omega'] * np.cos(args['w2']*t + args['phi'])
H = [H0, [H1, H1_coeff]]

psi0_g = tensor(basis(2,0), basis(2,0))
psi0_e = tensor(basis(2,1), basis(2,0))
# tlist = np.linspace(0, 1000, 5001)  # ns
tlist = np.arange(0, 2500, 1)  # ns if 0.5 is used, there is fast oscillation. why?
args = {'Omega':Omega, 'w2': w2, 'phi':phi}

# output_g = mesolve(H, psi0_g, tlist, [], [tensor(qeye(2), sigmaz()),tensor(qeye(2), sigmax()),tensor(qeye(2), sigmay())], args, progress_bar=True)  
# output_e = mesolve(H, psi0_e, tlist, [], [tensor(qeye(2), sigmaz()),tensor(qeye(2), sigmax()),tensor(qeye(2), sigmay())], args, progress_bar=True)  
U = propagator(H, tlist, [], args=args, progress_bar=True)
x_g = expect(tensor(qeye(2), sigmax()), U*psi0_g)
y_g = expect(tensor(qeye(2), sigmay()), U*psi0_g)
z_g = expect(tensor(qeye(2), sigmaz()), U*psi0_g)

x_e = expect(tensor(qeye(2), sigmax()), U*psi0_e)
y_e = expect(tensor(qeye(2), sigmay()), U*psi0_e)
z_e = expect(tensor(qeye(2), sigmaz()), U*psi0_e)

# plot
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax.plot(tlist, x_g, tlist,y_g, tlist, z_g)
ax.plot(tlist, x_e, tlist,y_e, tlist, z_e)
ax.set_xlabel('Time (ns)', fontsize=18)
ax.set_ylabel(r'$<\sigma_z>$', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

# plot in Bloch sphere
bloch = Bloch()
bloch.add_points([x_g, y_g, z_g])
bloch.add_points([x_e, y_e, z_e])
bloch.show()

In [ ]:
H0

# Two-level two-qubit with SFQ

In [ ]:
def H1_coeff(t, args):
    out = functools.reduce(operator.add, 
        [np.exp(-(t-args['tc']-i*2*pi/args['w2'])**2/2/args['tau']**2) 
         * (np.heaviside(t-i*2*pi/args['w2'],0) + np.heaviside(t+2*args['tc']-i*2*pi/args['w2'],0)) 
         for i in range(args['n'])])
    
    return out

# parameters
# w2 = 5 * 2 *pi # in GHz*2*pi 
tau = 2e-3 # in ns 
tc = 5*tau # in ns 
del_theta = 2*pi/360*2 # in rad
n = 90

# tlist_pulseTrain = np.linspace(0, n * 2*pi/w_01, 501)
# args = {'tau':tau, 'tc': tc, 'w_01': w_01,'n':n}

# parameters
w1 = 5.2 * 2 *pi # in GHz*2*pi 
w2 = 5 * 2 * pi  # target Q
# J = 0.003*2*pi
# Omega = 0.05*2*pi
# phi = 0 * 2 * pi

b = destroy(2)

# solve Lindblad master equation
H0 = w1*tensor(b.dag()*b, qeye(2)) + w2*tensor(qeye(2), b.dag()*b) \
    + J*(tensor(b.dag(), b) + tensor(b, b.dag()))
H1 =  del_theta/2/np.sqrt(2*pi)/tau*tensor((b.dag() + b), qeye(2))
H = [H0, [H1, H1_coeff]]

psi0_g = tensor(basis(2,0), basis(2,0))
psi0_e = tensor(basis(2,1), basis(2,0))
tlist = np.arange(0, np.round(n * 2*pi/w_01), 1)
# tlist = np.arange(0, 2500, 1)  # ns if 0.5 is used, there is fast oscillation. why?
args = {'tau':tau, 'tc': tc, 'w2': w2,'n':n}

# output_g = mesolve(H, psi0_g, tlist, [], [tensor(qeye(2), sigmaz()),tensor(qeye(2), sigmax()),tensor(qeye(2), sigmay())], args, progress_bar=True)  
# output_e = mesolve(H, psi0_e, tlist, [], [tensor(qeye(2), sigmaz()),tensor(qeye(2), sigmax()),tensor(qeye(2), sigmay())], args, progress_bar=True)  
U = propagator(H, tlist, [], args=args, progress_bar=True)
x_g = expect(tensor(qeye(2), sigmax()), U*psi0_g)
y_g = expect(tensor(qeye(2), sigmay()), U*psi0_g)
z_g = expect(tensor(qeye(2), sigmaz()), U*psi0_g)

x_e = expect(tensor(qeye(2), sigmax()), U*psi0_e)
y_e = expect(tensor(qeye(2), sigmay()), U*psi0_e)
z_e = expect(tensor(qeye(2), sigmaz()), U*psi0_e)

# plot
fig,ax = plt.subplots(1,1, figsize=(10,6))
ax.plot(tlist, x_g, tlist,y_g, tlist, z_g)
ax.plot(tlist, x_e, tlist,y_e, tlist, z_e)
ax.set_xlabel('Time (ns)', fontsize=18)
ax.set_ylabel(r'$<\sigma_z>$', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid('on')

# plot in Bloch sphere
bloch = Bloch()
bloch.add_points([x_g, y_g, z_g])
bloch.add_points([x_e, y_e, z_e])
bloch.show()

In [ ]:
np.round(n * 2*pi/w_01)

In [ ]:
plt.plot(H1_coeff(tlist,args))

In [ ]:
propagator?

In [ ]:
Options?